# 内容ブーステッド協調フィルタリング

In [1]:
import numpy as np
import pandas as pd

In [2]:
rating = [
    { "user_id": 1, "item_id": 2, "rating": 4},
    { "user_id": 1, "item_id": 3, "rating": 3},
    { "user_id": 1, "item_id": 4, "rating": 1},
    { "user_id": 1, "item_id": 5, "rating": 2},
    { "user_id": 1, "item_id": 7, "rating": 4},
    { "user_id": 2, "item_id": 1, "rating": 5},
    { "user_id": 2, "item_id": 2, "rating": 5},
    { "user_id": 2, "item_id": 3, "rating": 4},
    { "user_id": 2, "item_id": 5, "rating": 3},
    { "user_id": 2, "item_id": 6, "rating": 3},
    { "user_id": 2, "item_id": 7, "rating": 5},
    { "user_id": 2, "item_id": 8, "rating": 4},
    { "user_id": 3, "item_id": 1, "rating": 4},
    { "user_id": 3, "item_id": 3, "rating": 5},
    { "user_id": 3, "item_id": 4, "rating": 3},
    { "user_id": 3, "item_id": 5, "rating": 2},
    { "user_id": 3, "item_id": 8, "rating": 3},
    { "user_id": 4, "item_id": 2, "rating": 3},
    { "user_id": 4, "item_id": 4, "rating": 2},
    { "user_id": 4, "item_id": 5, "rating": 1},
    { "user_id": 4, "item_id": 6, "rating": 1},
    { "user_id": 4, "item_id": 7, "rating": 3},
    { "user_id": 5, "item_id": 1, "rating": 2},
    { "user_id": 5, "item_id": 2, "rating": 1},
    { "user_id": 5, "item_id": 3, "rating": 2},
    { "user_id": 5, "item_id": 4, "rating": 4},
    { "user_id": 5, "item_id": 6, "rating": 3},
    { "user_id": 5, "item_id": 7, "rating": 2},
]

In [3]:
rating_df = pd.DataFrame(rating)
rating_df.groupby("item_id")["rating"].describe()

,count,mean,std,min,25%,50%,75%,max
item_id,,,,,,,,
1,3.0,3.666667,1.527525,2.0,3.00,4.0,4.50,5.0
2,4.0,3.250000,1.707825,1.0,2.50,3.5,4.25,5.0
3,4.0,3.500000,1.290994,2.0,2.75,3.5,4.25,5.0
4,4.0,2.500000,1.290994,1.0,1.75,2.5,3.25,4.0
5,4.0,2.000000,0.816497,1.0,1.75,2.0,2.25,3.0
6,3.0,2.333333,1.154701,1.0,2.00,3.0,3.00,3.0
7,4.0,3.500000,1.290994,2.0,2.75,3.5,4.25,5.0
8,2.0,3.500000,0.707107,3.0,3.25,3.5,3.75,4.0


In [4]:
item = [
    {"item_id": 1, "type": "赤身"},
    {"item_id": 2, "type": "赤身"},
    {"item_id": 3, "type": "赤身"},
    {"item_id": 4, "type": "白身"},
    {"item_id": 5, "type": "白身"},
    {"item_id": 6, "type": "白身"},
    {"item_id": 7, "type": "光物"},
    {"item_id": 8, "type": "光物"},
    {"item_id": 9, "type": "光物"},
]
item_df = pd.DataFrame(item)
item_df.head()

,item_id,type
0,1,赤身
1,2,赤身
2,3,赤身
3,4,白身
4,5,白身


In [5]:
user_item = rating_df.merge(item_df, on="item_id", how="right")
train = user_item.copy()
matrix = user_item.pivot(index="user_id", columns="item_id", values="rating")
matrix.drop(matrix.index[0], inplace=True)
matrix

item_id,1,2,3,4,5,6,7,8,9
user_id,,,,,,,,,
1.0,NaN,4.0,3.0,1.0,2.0,NaN,4.0,NaN,NaN
2.0,5.0,5.0,4.0,NaN,3.0,3.0,5.0,4.0,NaN
3.0,4.0,NaN,5.0,3.0,2.0,NaN,NaN,3.0,NaN
4.0,NaN,3.0,NaN,2.0,1.0,1.0,3.0,NaN,NaN
5.0,2.0,1.0,2.0,4.0,NaN,3.0,2.0,NaN,NaN


In [6]:
user_item.head()

,user_id,item_id,rating,type
0,1.0,2,4.0,赤身
1,2.0,2,5.0,赤身
2,4.0,2,3.0,赤身
3,5.0,2,1.0,赤身
4,1.0,3,3.0,赤身


In [7]:
Ui = rating_df.groupby('user_id').agg({"item_id": list})["item_id"].to_dict()

In [8]:
train.drop(train.index[28], inplace=True)

In [9]:
# 評価したtypeの平均値で欠損値を埋める
for user_id in matrix.index:
    for item_id in matrix.columns:
        if ~np.isnan(matrix.loc[user_id, item_id]):
            continue
        groupby = train.groupby(["user_id", "type"])["rating"].mean()
        item_type = item_df[item_df["item_id"]==item_id]["type"]
        matrix.loc[user_id, item_id] = groupby[user_id][item_type].iloc[-1]

In [10]:
matrix

item_id,1,2,3,4,5,6,7,8,9
user_id,,,,,,,,,
1.0,3.5,4.0,3.0,1.0,2.0,1.5,4.0,4.0,4.0
2.0,5.0,5.0,4.0,3.0,3.0,3.0,5.0,4.0,4.5
3.0,4.0,4.5,5.0,3.0,2.0,2.5,3.0,3.0,3.0
4.0,3.0,3.0,3.0,2.0,1.0,1.0,3.0,3.0,3.0
5.0,2.0,1.0,2.0,4.0,3.5,3.0,2.0,2.0,2.0


In [11]:
# ユーザ同士の類似度計算
def sim(u, v):
    u_diff = u - np.mean(u)
    v_diff = v - np.mean(v)
    numerator = np.dot(u_diff, v_diff)
    denominator = np.sqrt(np.sum(u_diff**2))*np.sqrt(np.sum(v_diff**2))
    
    return round(numerator / denominator, 3)

In [12]:
S = np.zeros((len(matrix.index), len(matrix.index)))
user_id2index = dict(zip(matrix.index, range(len(matrix.index))))
for i in matrix.index:
    u_index = user_id2index[i]
    for j in matrix.index:
        v_index = user_id2index[j]
        S[u_index, v_index] = sim(matrix.loc[i,:].values, matrix.loc[j,:].values)

In [13]:
# 疑似相関行列　S
S

array([[ 1.   ,  0.887,  0.404,  0.828, -0.899],
       [ 0.887,  1.   ,  0.561,  0.848, -0.87 ],
       [ 0.404,  0.561,  1.   ,  0.683, -0.649],
       [ 0.828,  0.848,  0.683,  1.   , -0.772],
       [-0.899, -0.87 , -0.649, -0.772,  1.   ]])

In [14]:
# Sに重みを付与していく
alpha = 6
def calcu_w(len_i) -> float:
    w = len_i / alpha
    if (w > 1):
        w = 1
    return w

for i in matrix.index:
    Wi = calcu_w(len(Ui[i]))
    for j in matrix.index:
        Wj = calcu_w(len(Ui[j]))
            
        #調和平均
        w_hij = (2*Wi*Wj)/(Wi+Wj)
        w_sij = calcu_w(len(np.intersect1d(Ui[i], Ui[j])))
        if (w_sij > 1):
            w_sij = 1
        w_cij = (w_hij + w_sij) / 2
        
        S[user_id2index[i], user_id2index[j]] *= w_cij
S

array([[ 0.83333333,  0.69884848,  0.26933333,  0.621     , -0.70830303],
       [ 0.69884848,  1.        ,  0.442     ,  0.66812121, -0.7975    ],
       [ 0.26933333,  0.442     ,  0.83333333,  0.39841667, -0.45725   ],
       [ 0.621     ,  0.66812121,  0.39841667,  0.83333333, -0.60824242],
       [-0.70830303, -0.7975    , -0.45725   , -0.60824242,  1.        ]])

In [15]:
Ui

{1: [2, 3, 4, 5, 7],
 2: [1, 2, 3, 5, 6, 7, 8],
 3: [1, 3, 4, 5, 8],
 4: [2, 4, 5, 6, 7],
 5: [1, 2, 3, 4, 6, 7]}

In [16]:
# 類似ユーザK人選定
K_USERS = 3
Uu = {matrix.index[u]: {matrix.index[v]: round(S[u, v], 3) for v in user_id2index.values() if u!=v} for u in user_id2index.values()}
Uu = {u: dict(sorted(Uu[u].items(), key=lambda i:i[1], reverse=True)[:K_USERS]) for u in user_id2index.keys()}
Uu

{1.0: {2.0: 0.699, 4.0: 0.621, 3.0: 0.269},
 2.0: {1.0: 0.699, 4.0: 0.668, 3.0: 0.442},
 3.0: {2.0: 0.442, 4.0: 0.398, 1.0: 0.269},
 4.0: {2.0: 0.668, 1.0: 0.621, 3.0: 0.398},
 5.0: {3.0: -0.457, 4.0: -0.608, 1.0: -0.708}}

In [17]:
beta = 2
for u in matrix.index:
    presn_obj = Uu[u]
    wu_self = len(Ui[u])/alpha*beta
    if (len(Ui[u]) >=  alpha):
        wu_self = beta
    ru = np.mean(matrix.loc[u,:])
    for i in matrix.columns:
        if (i in Ui[u]):
            continue
        weight = wu_self*(matrix.loc[u,i]-ru)
        sum_v = np.mean([S[user_id2index[1],user_id2index[v]]*(matrix.loc[v,i]-np.mean(matrix.loc[v,:])) for v in Uu[u].keys()])
        sum_weight = wu_self + sum(Uu[u].values())
        
        rui = ru + (weight + sum_v)/sum_weight
        matrix.loc[u, i] = rui
        

In [18]:
matrix


item_id,1,2,3,4,5,6,7,8,9
user_id,,,,,,,,,
1.0,3.377248,4.000000,3.000000,1.000000,2.000000,2.041761,4.000000,3.534084,3.569860
2.0,5.000000,5.000000,4.000000,3.327315,3.000000,3.000000,5.000000,4.000000,4.356679
3.0,4.000000,4.258225,5.000000,3.000000,2.000000,2.544335,3.357541,3.000000,3.260503
4.0,2.836953,3.000000,2.763015,2.000000,1.000000,1.000000,3.000000,2.753712,2.788424
5.0,2.000000,1.000000,2.000000,4.000000,9.243516,3.000000,2.000000,-0.100402,0.078059


In [19]:
from collections import defaultdict
K_ITEMS = 2
pred_user2items = defaultdict(list)
for u in matrix.index:
    Ui_not = list(set(matrix.columns) ^ set(Ui[u]))
    pred_items = {i: matrix.loc[u,i] for i in Ui_not}
    pred_items = dict(sorted(pred_items.items(), key=lambda i:i[1], reverse=True)[:K_ITEMS]).keys()
    for i in pred_items:
        pred_user2items[u].append(i)

In [20]:
pred_user2items

defaultdict(list,
            {1.0: [9, 8], 2.0: [9, 4], 3.0: [2, 7], 4.0: [1, 9], 5.0: [5, 9]})